In [1]:
import numpy as np
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM

try:
    from colossus.cosmology import cosmology as ccosmo
    HAS_COLOSSUS = True
except ImportError:
    HAS_COLOSSUS = False


# -----------------------------
# Flamingo cosmology parameters
# -----------------------------
FLAMINGO = dict(
    H0=68.1,      # km/s/Mpc
    Om0=0.306,
    Ob0=0.0486,
    sigma8=0.807,
    ns=0.967,
    Tcmb0=2.7255, # K
)

def flamingo_h():
    return FLAMINGO["H0"] / 100.0

def make_astropy_flamingo_cosmo():
    return FlatLambdaCDM(
        H0=FLAMINGO["H0"] * u.km / u.s / u.Mpc,
        Om0=FLAMINGO["Om0"],
        Tcmb0=FLAMINGO["Tcmb0"] * u.K,
    )


# def f_sky_from_area_deg2(area_deg2: float) -> float:
#     """
#     Convert survey area in deg^2 to sky fraction f_sky.
#     Uses astropy.units if available; otherwise uses the exact conversion.
#     """
#     area_sr = (area_deg2 * u.deg**2).to_value(u.sr)
#     return area_sr / (4.0 * np.pi)

def f_sky_from_area_deg2(sky_area_deg2: float) -> float:
    """
    Convert survey area in deg^2 to sky fraction f_sky.
    Uses astropy.units if available; otherwise uses the exact conversion.
    """
    try:
        import astropy.units as u
        area_sr = (sky_area_deg2 * u.deg**2).to(u.sr).value
    except ImportError:
        # 1 deg = pi/180 rad => 1 deg^2 = (pi/180)^2 sr
        area_sr = sky_area_deg2 * (np.pi / 180.0) ** 2

    return area_sr / (4.0 * np.pi)

# ---------------------------------------------
# Method 1: "Simple" (Astropy comoving_volume)
# ---------------------------------------------
def volume_simple_astropy_comoving_volume(z_min, z_max, area_deg2, *, verbose=True):
    """
    Uses astropy comoving_volume (integrated exactly inside astropy).
    Returns V_survey in (Mpc/h)^3 and R_equiv in (Mpc/h).
    """
    cosmo = make_astropy_flamingo_cosmo()
    h = cosmo.H0.value / 100.0
    f_sky = f_sky_from_area_deg2(area_deg2)

    V_shell = (cosmo.comoving_volume(z_max) - cosmo.comoving_volume(z_min)) * f_sky  # Mpc^3
    V_Mpc3 = V_shell.to_value(u.Mpc**3)

    V = V_Mpc3 * h**3        # (Mpc/h)^3
    R = V ** (1.0 / 3.0)     # (Mpc/h)

    if verbose:
        print(f"[Simple/Astropy comoving_volume] V = {V:.9e} (Mpc/h)^3, R = {R:.6f} (Mpc/h)")

    return V, R


# ---------------------------------------------------
# Method 2: Astropy distance + analytic shell volume
# ---------------------------------------------------
def volume_astropy_distance_shell(z_min, z_max, area_deg2, *, verbose=True):
    """
    Computes r(z) from astropy comoving_distance, then uses V = 4/3*pi*(r_max^3-r_min^3).
    Returns V_survey in (Mpc/h)^3 and R_equiv in (Mpc/h).
    """
    cosmo = make_astropy_flamingo_cosmo()
    h = cosmo.H0.value / 100.0
    f_sky = f_sky_from_area_deg2(area_deg2)

    r_min_Mpc = cosmo.comoving_distance(z_min).to_value(u.Mpc)
    r_max_Mpc = cosmo.comoving_distance(z_max).to_value(u.Mpc)

    V_allsky_Mpc3 = (4.0 / 3.0) * np.pi * (r_max_Mpc**3 - r_min_Mpc**3)
    V_survey_Mpc3 = V_allsky_Mpc3 * f_sky

    V = V_survey_Mpc3 * h**3
    R = V ** (1.0 / 3.0)

    if verbose:
        print("-" * 80)
        print(f"[Astropy distance+shell] z={z_min:.3f}: chi={r_min_Mpc*h:.6f} (Mpc/h), {r_min_Mpc:.6f} (Mpc)")
        print(f"[Astropy distance+shell] z={z_max:.3f}: chi={r_max_Mpc*h:.6f} (Mpc/h), {r_max_Mpc:.6f} (Mpc)")
        print(f"[Astropy distance+shell] V = {V:.9e} (Mpc/h)^3, R = {R:.6f} (Mpc/h)")

    return V, R


# ----------------------------------------------------
# Method 3: Colossus distance + analytic shell volume
# ----------------------------------------------------
def get_colossus_flamingo_cosmo():
    params = {
        "flat": True,
        "H0": FLAMINGO["H0"],
        "Om0": FLAMINGO["Om0"],
        "Ob0": FLAMINGO["Ob0"],
        "sigma8": FLAMINGO["sigma8"],
        "ns": FLAMINGO["ns"],
    }
    # Guard for re-running in Jupyter notebooks
    try:
        ccosmo.addCosmology("flamingo", params)
    except Exception:
        pass
    return ccosmo.setCosmology("flamingo")


def volume_colossus_distance_shell(z_min, z_max, area_deg2, *, verbose=True):
    """
    Colossus comovingDistance returns Mpc/h directly.
    Returns V_survey in (Mpc/h)^3 and R_equiv in (Mpc/h).
    """
    if not HAS_COLOSSUS:
        raise ImportError("Colossus is not installed/importable in this environment.")

    cosmo = get_colossus_flamingo_cosmo()
    h = cosmo.h
    f_sky = f_sky_from_area_deg2(area_deg2)

    r_min = cosmo.comovingDistance(0.0, z_min)  # Mpc/h
    r_max = cosmo.comovingDistance(0.0, z_max)  # Mpc/h

    V_allsky = (4.0 / 3.0) * np.pi * (r_max**3 - r_min**3)  # (Mpc/h)^3
    V = V_allsky * f_sky
    R = V ** (1.0 / 3.0)

    if verbose:
        print("-" * 80)
        print(f"[Colossus distance+shell] z={z_min:.3f}: chi={r_min:.6f} (Mpc/h), {r_min/h:.6f} (Mpc)")
        print(f"[Colossus distance+shell] z={z_max:.3f}: chi={r_max:.6f} (Mpc/h), {r_max/h:.6f} (Mpc)")
        print(f"[Colossus distance+shell] V = {V:.9e} (Mpc/h)^3, R = {R:.6f} (Mpc/h)")

    return V, R


if __name__ == "__main__":
    # -----------------------------------------------------------------------------------
    # Main: DESY1 redMaPPer (observed) cluster richness + scaling to FLAMINGO (1 cGpc)^3
    # -----------------------------------------------------------------------------------
    # DESY1 redshift shell and footprint area (Table 1)
    # Reference: https://arxiv.org/pdf/2002.11124 
    z_lo, z_hi = 0.20, 0.35
    area_deg2 = 1437.0

    print("Flamingo cosmology (Astropy):")
    print(make_astropy_flamingo_cosmo())
    print("-" * 80)

    # FLAMINGO box: (1 cGpc)^3 (**no little h**)
    # In (Mpc/h), side length = 1000 * h, so V_box = (1000*h)^3 (Mpc/h)^3
    h = flamingo_h()
    V_box = (1000.0 * h) ** 3  # (Mpc/h)^3
    print(f"FLAMINGO box: L = 1000 cMpc, h={h:.3f} => L = {1000*h:.3f} (Mpc/h), V_box = {V_box:.9e} (Mpc/h)^3")
    print("-" * 80)

    # Three volume estimates (just for checking the values)
    V_simple, R_simple = volume_simple_astropy_comoving_volume(z_lo, z_hi, area_deg2, verbose=True)
    V_ast, R_ast       = volume_astropy_distance_shell(z_lo, z_hi, area_deg2, verbose=True)

    if HAS_COLOSSUS:
        V_col, R_col   = volume_colossus_distance_shell(z_lo, z_hi, area_deg2, verbose=True)
    else:
        V_col, R_col = None, None
        print("[Colossus] Not available (import failed). Install colossus to run this branch.")
    print("-" * 80)

    # Compare scaling factors (dimensionless): scale = V_box / V_survey
    scale_simple = V_box / V_simple
    scale_ast    = V_box / V_ast
    print(f"Scale factors (V_box / V_survey):")
    print(f"  simple (comoving_volume): {scale_simple:.9e}")
    print(f"  astropy (distance+shell): {scale_ast:.9e}")
    print("-" * 80)
    
    if V_col is not None:
        scale_col = V_box / V_col
        print(f"  colossus (distance+shell): {scale_col:.9e}")
        print(f"  ratio colossus / astropy(shell): {V_col / V_ast:.9e}")
        print(f"  ratio simple / astropy(shell):   {V_simple / V_ast:.9e}")
    print("-" * 80)

    # DESY1 richness (See Table 1)
    DESY1_lam = np.array([762, 376, 123, 91])

    # Expected richness in FLAMINGO box by volume scaling
    lam_simple = DESY1_lam * scale_simple
    lam_ast    = DESY1_lam * scale_ast

    print("Scaled richness in FLAMINGO box:")
    print("  simple:  ", lam_simple, "rounded:", np.round(lam_simple).astype(int))
    print("  astropy: " , lam_ast,   "rounded:", np.round(lam_ast).astype(int))

    if V_col is not None:
        lam_col = DESY1_lam * (V_box / V_col)
        print("  colossus:", lam_col, "rounded:", np.round(lam_col).astype(int))


Flamingo cosmology (Astropy):
FlatLambdaCDM(H0=68.1 km / (Mpc s), Om0=0.306, Tcmb0=2.7255 K, Neff=3.04, m_nu=[0. 0. 0.] eV, Ob0=None)
--------------------------------------------------------------------------------
FLAMINGO box: L = 1000 cMpc, h=0.681 => L = 681.000 (Mpc/h), V_box = 3.158212410e+08 (Mpc/h)^3
--------------------------------------------------------------------------------
[Simple/Astropy comoving_volume] V = 1.024594544e+08 (Mpc/h)^3, R = 467.933365 (Mpc/h)
--------------------------------------------------------------------------------
[Astropy distance+shell] z=0.200: chi=571.189759 (Mpc/h), 838.751481 (Mpc)
[Astropy distance+shell] z=0.350: chi=961.379902 (Mpc/h), 1411.717918 (Mpc)
[Astropy distance+shell] V = 1.024594544e+08 (Mpc/h)^3, R = 467.933365 (Mpc/h)
--------------------------------------------------------------------------------
[Colossus distance+shell] z=0.200: chi=571.189750 (Mpc/h), 838.751469 (Mpc)
[Colossus distance+shell] z=0.350: chi=961.379877 (Mpc